<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Study_Material_Generation_Based_on_Course_Ciriculumn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip  install -qU openai langchain PyPDF2 transformers tiktoken pinecone-client sentence-transformers qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.1 MB/s eta 0:00:00
 

## Packages for scanned documents

In [2]:
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (9,308 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 120831 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [3]:
! apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.2 [186 kB]
Fetched 186 kB in 0s (747 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 120878 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.2_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.2) ...
Setting up poppler-utils (22.02.0-2ubuntu0.2) ...
Processing triggers for man-db (2.10.2-1) ...


In [4]:
! pip install -U pytesseract

In [5]:
! pip install pdf2image

## Final Inference Script

## Using Langchain + Qdrant for Question Answering.

In [6]:
from qdrant_client import models, QdrantClient
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.qdrant import Qdrant
import PyPDF2
from PyPDF2 import PdfReader
from tqdm.auto import tqdm
from uuid import uuid4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
from time import time,sleep
import openai
from langchain import VectorDBQA
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
#
import os
import json
#
import io
import pytesseract
import shutil
import random
from PIL import Image
from pdf2image import convert_from_path
from configparser import ConfigParser

ai_for_world_config = ConfigParser()

#
def read_config(parser: ConfigParser, location: str) -> None:
    assert parser.read(location), f"Could not read config {location}"
#
def get_scores(sentence1,sentence2,smodel):
  # encode sentences to get their embeddings
  embedding1 = smodel.encode(sentence1, convert_to_tensor=True)
  embedding2 = smodel.encode(sentence2, convert_to_tensor=True)
  # compute similarity scores of two embeddings
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
  return cosine_scores.numpy().tolist()[0][0]
#
def is_scanned_pdf(file_path):
    page_num  = []
    page_content = []
    pdf_reader = PdfReader(file_path)
    text = ""
    for i,page in enumerate(pdf_reader.pages):
      page_num.append(f"Page{i+1}")
      page_content.append(page.extract_text())
      text += page.extract_text()

    # if no text found, it is likely a scanned pdf
    if text == "":
        print("Scanned PDF")
        return True,page_num,page_content

    else:
        print("Text-based PDF")
        return False,page_num,page_content
#
#
def convert_pdf_to_image(pdf_path):
    pages = convert_from_path(pdf_path, 350)
    #
    output_dir="pdf_img_dir"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    i = 1
    for page in pages:
        image_name = "Page_" + str(i) + ".jpg"
        image_path = os.path.join(output_dir,image_name)
        page.save(image_path, "JPEG")
        i = i+1
    return output_dir

# Extract text from image
#
def extract_text_from_image(image_path):
    extractedInformation = pytesseract.image_to_string(Image.open(image_path))
    return extractedInformation
#
#
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
                            chunk_size=1000,
                            chunk_overlap=200,
                            separators=["\n\n", "\n", " ", ""]
                            )
    chunks = text_splitter.split_text(text)
    return chunks
#
def preprocess_function(request_data,content_type=None):
  pdf_docs = request_data["pdf1"]
  prompt = request_data["query"]
  flag,page_num,page_content = is_scanned_pdf(pdf_docs)
  pdf_name = "temppdf"
  f_path = os.path.join("/tmp", pdf_name)
  #
  print(pdf_docs)
  if flag:
    '''
    with open(f_path, "wb") as f:
        f.write(pdf_docs.getbuffer())
    print(f"filesize : {os.path.getsize(f_path)}")
    '''
    page_num  = []
    page_content = []
    text = ""
    # convert pdf to image
    image_dir = convert_pdf_to_image(pdf_docs)
    for i,file in enumerate(os.listdir(image_dir)):
      page_num.append(f"Page{i+1}")
      image_path = os.path.join(image_dir,file)
      text += extract_text_from_image(image_path)
      page_content.append(extract_text_from_image(image_path))

      print(f"Text read from image: {text}")
  #
  metadatas = []
  texts = []
  for p,record in tqdm(zip(page_num,page_content)):
        # first get metadata fields for this record
        metadata = {'page number': p}
        # now we create chunks from the record text
        record_texts = get_text_chunks(record)
        # create individual metadata dicts for each chunk
        record_metadatas = [{
            "chunk": j, "text": text, **metadata
        } for j, text in enumerate(record_texts)]
        # append these to current batches
        texts.extend(record_texts)
        metadatas.extend(record_metadatas)
  c = []
  for i in range(len(metadatas)):
    c.append(metadatas[i]['text'])

  page_content = c
  return (flag,page_num,page_content,metadatas,prompt)
#
def model_load_function(model_path):
  embeddings = HuggingFaceEmbeddings(model_kwargs = {'device': 'cpu'},
                                    encode_kwargs = {'normalize_embeddings': False})
  smodel = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
  #
  return (embeddings,model_path,smodel)
#
def predict_function(context,model,content_type=None):
  embeddings,model_path,smodel= model
  flag,page_num,page_content,metadatas,prompt = context
  #Read config file for openai api key
  #
  '''
  CONFIG_FILE = os.path.join(model_path, "env.conf")
  read_config(ai_for_world_config, CONFIG_FILE)
  api_key = ai_for_world_config.get("openai", "api_key").strip()
  #define llm
  '''
  #
  llm = OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],temperature=0)
  #create vector store
  #

  doc_store = Qdrant.from_texts(page_content,
                              metadatas=metadatas,
                              embedding=embeddings,
                              location=":memory:",
                              prefer_grpc=True,

                              collection="doc_search")

  #query vector store
  prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end.
  Please provide an answer which is factually correct and based on the information retrieved from the vector store.

{context}

QUESTION:```{question}```
ANSWER:
  """
  PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context","question"]
  )
  #
  chain_type_kwargs = {"prompt": PROMPT}
  '''
  qa = RetrievalQA.from_chain_type(llm=OpenAI(),
                                   chain_type="stuff",
                                   retriever=docsearch.as_retriever(),
                                   return_source_documents=True)


  '''
  #get scores
  #similar_items = doc_store.similarity_search_with_relevance_scores(prompt)
  #print(similar_items)
  #
  qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name='gpt-3.5-turbo-16k',
                                                  openai_api_key=os.environ["OPENAI_API_KEY"],
                                                  temperature=0),
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt": PROMPT},
                                  retriever=doc_store.as_retriever(search_kwargs={"k": 5}),
                                      return_source_documents=True
                                  )



  # Serach for answer
  print(prompt)
  result = qa(prompt)
  return (result,smodel)
#
def postprocess_function(res, content_type=None):
  result,smodel = res
  query = result['query']
  answer = result['result']
  referrences = []
  pages = []
  #
  for i in range(len(result["source_documents"])):
    print(f"Content : {result['source_documents'][i].page_content}")
    referrences.append(result['source_documents'][i].page_content)
    print(f"Page Number : {result['source_documents'][i].metadata['page number']}")
    pages.append(result['source_documents'][i].metadata['page number'])


  # delete temp directories if any created
  scores = []
  for item in referrences:
    scores.append(get_scores(query,item,smodel))

  # Format the response
  predictions = {"Query":query,
                 "Answer":answer,
                 "Referrences":referrences,
                 "Page Numbers":pages,
                 "Scores": scores}

  output_dir="pdf_img_dir"
  if os.path.exists(output_dir):
    for  file in os.listdir(output_dir):
      file_name = os.path.join(output_dir,file)
      if os.path.exists(file_name) and os.path.isfile(file_name):
        print(f"File {file_name} exists. Hence Removing")
        os.remove(file_name)
  return json.dumps({"response": predictions})


In [7]:
import openai
from getpass import getpass
import os
os.environ["OPENAI_API_KEY"] =getpass()

··········


In [8]:
%%time
from langchain import PromptTemplate
from sentence_transformers import SentenceTransformer, util
request_data = {"pdf1":"/content/MMPAGE7.pdf",
                "query":"Please generate a detailed study material for the student for the topic Motion and Measurement which is factually correct and based on the information retrieved from the vector store. Also provide detailed examples for the concepts explained in a step by step manner."}
content = preprocess_function(request_data,content_type=None)

Scanned PDF
/content/MMPAGE7.pdf
Text read from image: ©

Overview

©

Types of
Motion

©

Measurement
of Length

 

 

Activity — 2: Measure diameter of a sphere

Wooden block method: Place the sphere
between two blocks in contact with a ruler as
shown in figure below. Read the distance
between the two blocks on the ruler accurately.
(The line of sight should be vertical.)

 

ACTIVITY — 3: Measure thickness of a coin

Take 10 similar 5 rupee coins and place them one above
the other to form a heap. Measure the combined
thickness of the coins. Divide this by 10 to obtain the
thickness of one coin. We should be careful to select units
of length in such a way that the numerical value of

measurement is as easy as possible.

 

 

 

Activity — 4: Measure diameter of a wire

 

Take a wire whose diameter is to be measured. Take a thick pencil and
wind 20 turns of wire around it. Make sure that the wire is wound
tightly and there are no gaps between the windings. Take a ruler and
measure t

0it [00:00, ?it/s]

CPU times: user 4.06 s, sys: 219 ms, total: 4.28 s
Wall time: 18.4 s


In [9]:
flag,page_num,page_content,metadatas,prompt = content
page_content

['©\n\nOverview\n\n©\n\nTypes of\nMotion\n\n©\n\nMeasurement\nof Length\n\n \n\n \n\nActivity — 2: Measure diameter of a sphere\n\nWooden block method: Place the sphere\nbetween two blocks in contact with a ruler as\nshown in figure below. Read the distance\nbetween the two blocks on the ruler accurately.\n(The line of sight should be vertical.)\n\n \n\nACTIVITY — 3: Measure thickness of a coin\n\nTake 10 similar 5 rupee coins and place them one above\nthe other to form a heap. Measure the combined\nthickness of the coins. Divide this by 10 to obtain the\nthickness of one coin. We should be careful to select units\nof length in such a way that the numerical value of\n\nmeasurement is as easy as possible.\n\n \n\n \n\n \n\nActivity — 4: Measure diameter of a wire',
 'measurement is as easy as possible.\n\n \n\n \n\n \n\nActivity — 4: Measure diameter of a wire\n\n \n\nTake a wire whose diameter is to be measured. Take a thick pencil and\nwind 20 turns of wire around it. Make sure that t

In [10]:
page_num

['Page1']

In [11]:
%%time
model = model_load_function(model_path=None)


CPU times: user 4.4 s, sys: 3.31 s, total: 7.71 s
Wall time: 47.7 s


In [12]:
%%time
response = predict_function(content,model,content_type=None)

Please generate a detailed study material for the student for the topic Motion and Measurement which is factually correct and based on the information retrieved from the vector store. Also provide detailed examples for the concepts explained in a step by step manner.
CPU times: user 2.17 s, sys: 47.9 ms, total: 2.21 s
Wall time: 42 s


In [13]:
response

({'query': 'Please generate a detailed study material for the student for the topic Motion and Measurement which is factually correct and based on the information retrieved from the vector store. Also provide detailed examples for the concepts explained in a step by step manner.',
  'result': 'Study Material: Motion and Measurement\n\nOverview:\nMotion is the change in position of an object with respect to its surroundings. It can be described in terms of distance, displacement, speed, and velocity. Measurement is the process of determining the size, length, or amount of something using appropriate units.\n\nTypes of Motion:\n1. Translational Motion: This type of motion involves the movement of an object from one place to another without any rotation. Examples include a car moving on a straight road or a ball rolling down a hill.\n2. Rotational Motion: This type of motion involves the movement of an object around an axis or a fixed point. Examples include the rotation of a wheel or the

In [14]:
response[0]['result']

'Study Material: Motion and Measurement\n\nOverview:\nMotion is the change in position of an object with respect to its surroundings. It can be described in terms of distance, displacement, speed, and velocity. Measurement is the process of determining the size, length, or amount of something using appropriate units.\n\nTypes of Motion:\n1. Translational Motion: This type of motion involves the movement of an object from one place to another without any rotation. Examples include a car moving on a straight road or a ball rolling down a hill.\n2. Rotational Motion: This type of motion involves the movement of an object around an axis or a fixed point. Examples include the rotation of a wheel or the spinning of a top.\n3. Vibrational Motion: This type of motion involves the back-and-forth movement of an object around a fixed position. Examples include the oscillation of a pendulum or the vibration of a guitar string.\n\nMeasurement of Length:\nLength is a fundamental quantity used to mea

## AI Generated Content

In [18]:
print(response[0]['result'])

Study Material: Motion and Measurement

Overview:
Motion is the change in position of an object with respect to its surroundings. It can be described in terms of distance, displacement, speed, and velocity. Measurement is the process of determining the size, length, or amount of something using appropriate units.

Types of Motion:
1. Translational Motion: This type of motion involves the movement of an object from one place to another without any rotation. Examples include a car moving on a straight road or a ball rolling down a hill.
2. Rotational Motion: This type of motion involves the movement of an object around an axis or a fixed point. Examples include the rotation of a wheel or the spinning of a top.
3. Vibrational Motion: This type of motion involves the back-and-forth movement of an object around a fixed position. Examples include the oscillation of a pendulum or the vibration of a guitar string.

Measurement of Length:
Length is a fundamental quantity used to measure the siz

In [15]:
%%time
out = postprocess_function(response, content_type=None)

Content : ©

Overview

©

Types of
Motion

©

Measurement
of Length

 

 

Activity — 2: Measure diameter of a sphere

Wooden block method: Place the sphere
between two blocks in contact with a ruler as
shown in figure below. Read the distance
between the two blocks on the ruler accurately.
(The line of sight should be vertical.)

 

ACTIVITY — 3: Measure thickness of a coin

Take 10 similar 5 rupee coins and place them one above
the other to form a heap. Measure the combined
thickness of the coins. Divide this by 10 to obtain the
thickness of one coin. We should be careful to select units
of length in such a way that the numerical value of

measurement is as easy as possible.

 

 

 

Activity — 4: Measure diameter of a wire
Page Number : Page1
Content : measurement is as easy as possible.

 

 

 

Activity — 4: Measure diameter of a wire

 

Take a wire whose diameter is to be measured. Take a thick pencil and
wind 20 turns of wire around it. Make sure that the wire is wound
tightl

In [16]:
out

'{"response": {"Query": "Please generate a detailed study material for the student for the topic Motion and Measurement which is factually correct and based on the information retrieved from the vector store. Also provide detailed examples for the concepts explained in a step by step manner.", "Answer": "Study Material: Motion and Measurement\\n\\nOverview:\\nMotion is the change in position of an object with respect to its surroundings. It can be described in terms of distance, displacement, speed, and velocity. Measurement is the process of determining the size, length, or amount of something using appropriate units.\\n\\nTypes of Motion:\\n1. Translational Motion: This type of motion involves the movement of an object from one place to another without any rotation. Examples include a car moving on a straight road or a ball rolling down a hill.\\n2. Rotational Motion: This type of motion involves the movement of an object around an axis or a fixed point. Examples include the rotation

In [17]:
print(out)

{"response": {"Query": "Please generate a detailed study material for the student for the topic Motion and Measurement which is factually correct and based on the information retrieved from the vector store. Also provide detailed examples for the concepts explained in a step by step manner.", "Answer": "Study Material: Motion and Measurement\n\nOverview:\nMotion is the change in position of an object with respect to its surroundings. It can be described in terms of distance, displacement, speed, and velocity. Measurement is the process of determining the size, length, or amount of something using appropriate units.\n\nTypes of Motion:\n1. Translational Motion: This type of motion involves the movement of an object from one place to another without any rotation. Examples include a car moving on a straight road or a ball rolling down a hill.\n2. Rotational Motion: This type of motion involves the movement of an object around an axis or a fixed point. Examples include the rotation of a wh

In [ ]:
out.split("Referrences:")[-1]

'{"response": {"Query": "what is a neccessary Part of the Capital Structure according to Fred Wilson?", "Answer": "According to Fred Wilson, a necessary part of the capital structure is employee equity or ESOPs (Employee Stock Ownership Plans). He states, \\"I can\'t think of a term sheet that we have issued that didn\'t have a specific provision for employee equity.\\"", "Referrences": ["A Necessary Part of the Capital Structure  \\n\\u2022Venture capitalists require ESOPs . For many VCs, establishing a \\nstock option pool is a prerequisite to closing a deal  \\n\\u2013In an industry where options are ubiquitous, startups are compelled \\nto offer options packages to compete for top talent with other \\nventure -backed companies  \\n\\u2013When operating budgets are tight, competitive compensation \\npackages may not be possible; options can be used to incentivize \\nemployees instead of cash  \\nI can\'t think of a term sheet that we have issued that didn\'t have a \\nspecific provi

In [ ]:
from typing import Dict, List, Optional, Tuple
from langchain.docstore.document import Document
from langchain.vectorstores import Qdrant
import pandas as pd
import numpy as np

In [ ]:
class QdrantWrapper(Qdrant):
    query_text_to_document_score_tuples: Dict[str, List[Tuple[Document, float]]] = {}

    def similarity_search_with_score(
        self,
        query: str,
        k: int = 4,
        filter: Optional[dict] = None,
        namespace: Optional[str] = None,
    ) -> List[Tuple[Document, float]]:
        document_score_tuples = super().similarity_search_with_score(query=query)
        self.query_text_to_document_score_tuples[query] = document_score_tuples
        return document_score_tuples

    @property
    def retrieval_dataframe(self) -> pd.DataFrame:
        query_texts = []
        document_texts = []
        retrieval_ranks = []
        scores = []
        for query_text, document_score_tuples in self.query_text_to_document_score_tuples.items():
            for retrieval_rank, (document, score) in enumerate(document_score_tuples):
                query_texts.append(query_text)
                document_texts.append(document.page_content)
                retrieval_ranks.append(retrieval_rank)
                scores.append(score)
        return pd.DataFrame.from_dict(
            {
                "query_text": query_texts,
                "document_text": document_texts,
                "retrieval_rank": retrieval_ranks,
                "score": scores,
            }
        )

In [ ]:
d = QdrantWrapper.from_texts(page_content,
                              metadatas=metadatas,
                              embedding=HuggingFaceEmbeddings(),
                              location=":memory:",
                              collection="doc_search",
                              )

In [ ]:
#query vector store
prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
  Please provide an answer which is factually correct and based on the information retrieved from the vector store.
  Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
  """
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context","question"]
  )

In [ ]:
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name='gpt-3.5-turbo-16k',
                                                  openai_api_key=os.environ["OPENAI_API_KEY"],
                                                  temperature=0),
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt": PROMPT},
                                  retriever=d.as_retriever(),
                                  return_source_documents=True
                                  )

In [ ]:
response_text = qa("what is a neccessary Part of the Capital Structure according to Fred Wilson?")
retrievals_df = d.retrieval_dataframe.tail(5)
contexts = retrievals_df["document_text"].to_list()
scores = retrievals_df["score"].to_list()
query_embedding = embeddings.query_embedding_dataframe["text_vector"].iloc[-1]

print("Response")
print("========")
print()

TypeError: ignored

https://towardsdatascience.com/semantic-similarity-using-transformers-8f3cb5bf66d6

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [ ]:
smodel = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [ ]:
def scores(sentence1,sentence2,smodel):
  # encode sentences to get their embeddings
  embedding1 = smodel.encode(sentence1, convert_to_tensor=True)
  embedding2 = smodel.encode(sentence2, convert_to_tensor=True)
  # compute similarity scores of two embeddings
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
  return cosine_scores.numpy().tolist()[0][0]

In [ ]:
txt1 = "what is a neccessary Part of the Capital Structure according to Fred Wilson?"
txt2 = """A Necessary Part of the Capital Structure
•Venture capitalists require ESOPs . For many VCs, establishing a
stock option pool is a prerequisite to closing a deal
–In an industry where options are ubiquitous, startups are compelled
to offer options packages to compete for top talent with other
venture -backed companies
–When operating budgets are tight, competitive compensation
packages may not be possible; options can be used to incentivize
employees instead of cash
I can't think of a term sheet that we have issued that didn't have a
specific provision for employee equity.
Fred Wilson, Union Square Ventures
Page Number : Page10
Content : The Top -Down Process
1.Determine how much equity to set aside for non -founder employees
2.Create a schedule of how this equity will be distributed over time"""

In [ ]:
scores(txt1,txt2)

0.47183188796043396

In [ ]:
scores(txt1,txt2).numel()

1

In [ ]:
database_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/langchain-pinecone/database.parquet"
)
database_df.head()

,text,text_vector
0,\nAccess tutorials of what's possible with Ari...,"[-0.0036313518733896957, 0.0026182831234325553..."
1,"\n{% hint style=""info"" %}\nYour model type det...","[-0.006279695251247202, -0.0005314257337369415..."
2,\nExamples for logging explainability metrics....,"[-0.007306394403318994, 0.0007199545863166625,..."
3,\n| Google Cloud Services | Link |\n| ------...,"[0.002060703379662884, -0.0037287262540240764,..."
4,\n| Sending 10 Million Inferences to Arize in ...,"[-0.008325811390756237, -0.008001045242455413,..."


In [ ]:

query_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/langchain-pinecone/langchain_pinecone_query_dataframe_with_user_feedbackv2.parquet"
)
query_df.head()

,text,text_vector,response,context_text_0,context_text_1,context_similarity_0,context_similarity_1,user_feedback
0,How do I use the SDK to upload a ranking model?,"[-0.009508815594017506, 0.008546245284378529, ...","To use the SDK to upload a ranking model, you ...",\n**Check out our** **How to Monitor Ranking M...,"\n**Use the 'arize-demo-hotel-ranking' model, ...",0.827150,0.819964,-1.0
1,What drift metrics are supported in Arize?,"[-0.009323584847152233, -0.0005137172411195934...",Arize supports several drift metrics including...,\nArize calculates drift metrics such as Popul...,"\nDrift monitors measure distribution drift, w...",0.886113,0.884156,NaN
2,Does Arize support batch models?,"[-0.016475794836878777, -0.0009410726488567889...","Yes, Arize supports batch models as long as th...",\nArize supports many model types - check out ...,\nArize natively supports binary classificatio...,0.870960,0.856241,-1.0
3,Does Arize support training data?,"[-0.02083049900829792, -0.008836966007947922, ...",Arize is primarily a platform for model perfor...,\nArize natively supports binary classificatio...,\nArize natively supports tabular/structured d...,0.869958,0.865478,-1.0
4,How do I configure a threshold if my data has ...,"[-0.0008055078214965761, -0.011270225048065186...","If your data has seasonality trends, you may w...",\nAutomatic thresholds set a dynamic threshold...,\nAutothresholds are calculated based on a sta...,0.831995,0.823832,-1.0


In [ ]:

database_centroid = database_df["text_vector"].mean()
database_df["centered_text_vector"] = database_df["text_vector"].apply(
    lambda x: x - database_centroid
)
query_centroid = query_df["text_vector"].mean()
query_df["centered_text_vector"] = query_df["text_vector"].apply(lambda x: x - query_centroid)

In [ ]:
query_df.head()

,text,text_vector,response,context_text_0,context_text_1,context_similarity_0,context_similarity_1,user_feedback,centered_text_vector
0,How do I use the SDK to upload a ranking model?,"[-0.009508815594017506, 0.008546245284378529, ...","To use the SDK to upload a ranking model, you ...",\n**Check out our** **How to Monitor Ranking M...,"\n**Use the 'arize-demo-hotel-ranking' model, ...",0.827150,0.819964,-1.0,"[-0.005765025750840436, 0.016808218208686464, ..."
1,What drift metrics are supported in Arize?,"[-0.009323584847152233, -0.0005137172411195934...",Arize supports several drift metrics including...,\nArize calculates drift metrics such as Popul...,"\nDrift monitors measure distribution drift, w...",0.886113,0.884156,NaN,"[-0.005579795003975163, 0.0077482556831883435,..."
2,Does Arize support batch models?,"[-0.016475794836878777, -0.0009410726488567889...","Yes, Arize supports batch models as long as th...",\nArize supports many model types - check out ...,\nArize natively supports binary classificatio...,0.870960,0.856241,-1.0,"[-0.012732004993701706, 0.007320900275451148, ..."
3,Does Arize support training data?,"[-0.02083049900829792, -0.008836966007947922, ...",Arize is primarily a platform for model perfor...,\nArize natively supports binary classificatio...,\nArize natively supports tabular/structured d...,0.869958,0.865478,-1.0,"[-0.01708670916512085, -0.0005749930836399849,..."
4,How do I configure a threshold if my data has ...,"[-0.0008055078214965761, -0.011270225048065186...","If your data has seasonality trends, you may w...",\nAutomatic thresholds set a dynamic threshold...,\nAutothresholds are calculated based on a sta...,0.831995,0.823832,-1.0,"[0.0029382820216804937, -0.0030082521237572487..."


In [ ]:

import os
import textwrap
from datetime import timedelta
from typing import Dict, List, Optional, Tuple
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
from tqdm import tqdm

pd.set_option("display.max_colwidth", None)
#
EVALUATION_SYSTEM_MESSAGE = "You will be given a query and a reference text. You must determine whether the reference text contains an answer to the input query. Your response must be binary (0 or 1) and should not contain any text or characters aside from 0 or 1. 0 means that the reference text does not contain an answer to the query. 1 means the reference text contains an answer to the query."
QUERY_CONTEXT_PROMPT_TEMPLATE = """# Query: {query}

# Reference: {reference}

# Binary: """

openai.api_key = os.environ["OPENAI_API_KEY"]
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def evaluate_query_and_retrieved_context(query: str, context: str, model_name: str) -> str:
    prompt = QUERY_CONTEXT_PROMPT_TEMPLATE.format(
        query=query,
        reference=context,
    )
    response = openai.ChatCompletion.create(
        messages=[
            {"role": "system", "content": EVALUATION_SYSTEM_MESSAGE},
            {"role": "user", "content": prompt},
        ],
        model=model_name,
    )
    return response["choices"][0]["message"]["content"]


def evaluate_retrievals(
    retrievals_data: Dict[str, str],
    model_name: str,
) -> List[str]:
    responses = []
    for query, retrieved_context in tqdm(retrievals_data.items()):
        response = evaluate_query_and_retrieved_context(query, retrieved_context, model_name)
        responses.append(response)
    return responses


def process_binary_responses(
    binary_responses: List[str], binary_to_string_map: Dict[int, str]
) -> List[str]:
    """
    Parse binary responses and convert to the desired format
    converts them to the desired format. The binary_to_string_map parameter
    should be a dictionary mapping binary values (0 or 1) to the desired
    string values (e.g. "irrelevant" or "relevant").
    """
    processed_responses = []
    for binary_response in binary_responses:
        try:
            binary_value = int(binary_response.strip())
            processed_response = binary_to_string_map[binary_value]
        except (ValueError, KeyError):
            processed_response = None
        processed_responses.append(processed_response)
    return processed_responses


sample_query_df = query_df.head(10).copy()
evaluation_model_name = "gpt-4"  # use GPT-4 if you have access
for context_index in range(2):
    retrievals_data = {
        row["text"]: row[f"context_text_{context_index}"] for _, row in sample_query_df.iterrows()
    }
    raw_responses = evaluate_retrievals(retrievals_data, evaluation_model_name)
    processed_responses = process_binary_responses(raw_responses, {0: "irrelevant", 1: "relevant"})
    sample_query_df[f"openai_relevance_{context_index}"] = processed_responses
sample_query_df[
    ["text", "context_text_0", "openai_relevance_0", "context_text_1", "openai_relevance_1"]
].head(10)

100%|██████████| 10/10 [00:06<00:00,  1.59it/s]


,text,context_text_0,openai_relevance_0,context_text_1,openai_relevance_1
0,How do I use the SDK to upload a ranking model?,\n**Check out our** **How to Monitor Ranking Model's** **blog and follow along with our various** **Colab** **examples here.**&#x20;\n\n,irrelevant,"\n**Use the 'arize-demo-hotel-ranking' model, available in all free accounts, to follow along.**&#x20;\n\n",irrelevant
1,What drift metrics are supported in Arize?,"\nArize calculates drift metrics such as Population Stability Index, KL Divergence, and Wasserstein Distance. Arize computes drift by measuring distribution changes between the model’s production values and a baseline (reference dataset). Users can configure a baseline to be any time window of a:\n\n1. Pre-production dataset (training, test, validation) or\n2. Fixed or moving time period from production (e.g. last 30 days, last 60 days).&#x20;\n\nBaselines are saved in Arize so that users can compare several versions and/or environments against each other across moving or fixed time windows. For more details on baselines, visit here.\n\n",relevant,"\nDrift monitors measure distribution drift, which is the difference between two statistical distributions.&#x20;\n\nArize offers various distributional drift metrics to choose from when setting up a monitor. Each metric is tailored to a specific use case; refer to this guide to help choose the appropriate metric for various ML use cases.\n\n",relevant
2,Does Arize support batch models?,\nArize supports many model types - check out our various Model Types to learn more.&#x20;\n\n,irrelevant,"\nArize natively supports binary classification, multi-class classification, regression, ranking, NLP, and CV model types. Your model type informs the **data ingestion format** and **the performance metrics** that can be utilized in the platform.&#x20;\n\n",irrelevant
3,Does Arize support training data?,"\nArize natively supports binary classification, multi-class classification, regression, ranking, NLP, and CV model types. Your model type informs the **data ingestion format** and **the performance metrics** that can be utilized in the platform.&#x20;\n\n",irrelevant,"\nArize natively supports tabular/structured data types (strings, floats, booleans, etc), as well as embedding support for NLP, Image, and other unstructured data types.\n\n",irrelevant
4,How do I configure a threshold if my data has seasonality trends?,\nAutomatic thresholds set a dynamic threshold value for each data point. Arize generates an auto threshold when there are at least 14 days of production data to determine a trend. You can edit your auto threshold sensitivity by changing the standard deviation number in the 'Monitor Settings' card.&#x20;\n\nLearn more here about how an auto threshold value is calculated.&#x20;\n\nAuto Threshold\n\nToggle automatic thresholds on or off from the “Edit monitor” configuration.\n\n,irrelevant,"\nAutothresholds are calculated based on a statistical analysis of data over 14 days. Each day, a data point is collected, and after 14 days, the average (mean) and standard deviation of these data points are computed. The thresholds is then set by adding or subtracting the standard deviation from the average.\n\n",irrelevant
5,How are clusters in the UMAP calculated? When are the clusters refreshed?,"\n1. **Select your data and Generate UMAP**\n\n * Select a point on the drift visualization at the top of the page and generate UMAP to visualize the point in time selected\n\n \n\n\n2. **Investigate your worst drifted clusters**\n\n * Clusters are groups of related points in the point cloud. The closer points or clusters are to each other, the more similar they are. Clusters allow you to easily visualize data that differs from your baseline but also allows you to visualize the global and local structure of your data.\n\n You can select a specific cluster to further investigate and view the data associated with the cluster\n3. **Investigate the data points that belong in

In [ ]:
openai_evaluations_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/langchain-pinecone/langchain_pinecone_openai_evaluations.parquet",
)[["text", "openai_relevance_0", "openai_relevance_1"]]
query_df = pd.merge(query_df, openai_evaluations_df, on="text")
query_df[["text", "context_text_0", "context_text_1", "openai_relevance_0", "openai_relevance_1"]]

,text,context_text_0,context_text_1,openai_relevance_0,openai_relevance_1
0,How do I use the SDK to upload a ranking model?,\n**Check out our** **How to Monitor Ranking Model's** **blog and follow along with our various** **Colab** **examples here.**&#x20;\n\n,"\n**Use the 'arize-demo-hotel-ranking' model, available in all free accounts, to follow along.**&#x20;\n\n",irrelevant,irrelevant
1,What drift metrics are supported in Arize?,"\nArize calculates drift metrics such as Population Stability Index, KL Divergence, and Wasserstein Distance. Arize computes drift by measuring distribution changes between the model’s production values and a baseline (reference dataset). Users can configure a baseline to be any time window of a:\n\n1. Pre-production dataset (training, test, validation) or\n2. Fixed or moving time period from production (e.g. last 30 days, last 60 days).&#x20;\n\nBaselines are saved in Arize so that users can compare several versions and/or environments against each other across moving or fixed time windows. For more details on baselines, visit here.\n\n","\nDrift monitors measure distribution drift, which is the difference between two statistical distributions.&#x20;\n\nArize offers various distributional drift metrics to choose from when setting up a monitor. Each metric is tailored to a specific use case; refer to this guide to help choose the appropriate metric for various ML use cases.\n\n",relevant,irrelevant
2,Does Arize support batch models?,\nArize supports many model types - check out our various Model Types to learn more.&#x20;\n\n,"\nArize natively supports binary classification, multi-class classification, regression, ranking, NLP, and CV model types. Your model type informs the **data ingestion format** and **the performance metrics** that can be utilized in the platform.&#x20;\n\n",irrelevant,irrelevant
3,Does Arize support training data?,"\nArize natively supports binary classification, multi-class classification, regression, ranking, NLP, and CV model types. Your model type informs the **data ingestion format** and **the performance metrics** that can be utilized in the platform.&#x20;\n\n","\nArize natively supports tabular/structured data types (strings, floats, booleans, etc), as well as embedding support for NLP, Image, and other unstructured data types.\n\n",irrelevant,irrelevant
4,How do I configure a threshold if my data has seasonality trends?,\nAutomatic thresholds set a dynamic threshold value for each data point. Arize generates an auto threshold when there are at least 14 days of production data to determine a trend. You can edit your auto threshold sensitivity by changing the standard deviation number in the 'Monitor Settings' card.&#x20;\n\nLearn more here about how an auto threshold value is calculated.&#x20;\n\nAuto Threshold\n\nToggle automatic thresholds on or off from the “Edit monitor” configuration.\n\n,"\nAutothresholds are calculated based on a statistical analysis of data over 14 days. Each day, a data point is collected, and after 14 days, the average (mean) and standard deviation of these data points are computed. The thresholds is then set by adding or subtracting the standard deviation from the average.\n\n",irrelevant,irrelevant
...,...,...,...,...,...
148,Do you support IoU for image segmentation?,\nArize supports 2 methods for ingesting and visualizing feature importance\n\nMethodUser Calculated SHAPshap.mdimage (33).pngGroup 1 (2).pngSurrogate Modelsurrogate-model.mdimage (23).pngGroup 2 (2).png\n\n\n\n,"\nObject detection models identify and locate objects within images or videos by assigning them specific bounding boxes.\n\nApplicable Metrics: Accuracy, Euclidian Distance (embeddings)\n\nClick here for all valid model types and metric combinations.&#x20;\n\n",irrelevant,irrelevant
149,This is a test question?,"\nIn the example above, there wasn't enough context on video quality to be able to correctly answer the user questions. Adding more context can help.&#x20;\